# Week 10 Assignment Review

Some submissions that met all of the requirements of the assignment:

- Zhijing Zhang: https://github.com/steinszzh/DAV-5400/blob/master/week10/zh_zhang_W10_assn.ipynb


- Sara Ferrari: https://github.com/saramariaferrari/DAV-5400/blob/master/S_Ferrari_W10_assn.ipynb


- Michael Vizelman: https://github.com/vizelman/DAV-5400/blob/master/M_Vizelman_W10_assn.ipynb


- Manling Yang: https://github.com/mandy941/week9/blob/master/M_Yang_W10_assn.ipynb

# Week 11: Data Aggregation & Grouping

## GroupBy: How it works in Pandas

- Pandas '__groupby()__' function creates __subsets__ of data on the basis of one or more grouping keys (e.g., two column names within a DataFrame)


- Data contained within a Pandas DataFrame can be grouped by rows or columns


- Desired computations are then applied to the results of GroupBy operations: each group is treated separately from any others that might have been derived via the GroupBy operation.


- Finally, the results of the separate computations are combined into a single data object.


- This process is often referred to as __split-apply-combine__


- Determining how you want your data grouped is the first step in the process.


An example from the PfDA text:

In [1]:
import numpy as np
import pandas as pd

In [2]:
# generate some test data
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.013753,0.039333
1,a,two,-0.132694,-1.667914
2,b,one,-0.614490,0.180166
3,b,two,-0.818761,0.014501
4,a,one,-0.444192,0.266835


In [3]:
# group the data based on the contents of the 'key1' column: since we have two data values
# within the column ('a' and 'b'), we will have two groupings created
grouped = df['data1'].groupby(df['key1'])

# display summary statistics for the groupings
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,-0.530213,0.446784,-1.013753,-0.728973,-0.444192,-0.288443,-0.132694
b,2.0,-0.716625,0.144442,-0.818761,-0.767693,-0.716625,-0.665557,-0.614490


In [4]:
# display just the mean values of the two groupings
grouped.mean()

key1
a   -0.530213
b   -0.716625
Name: data1, dtype: float64

In [5]:
# grouping by both a primary key AND a secondary key: Now we will have 4 separate groupings-
# a/one, a/two, b/one, and b/two
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.728973
      two    -0.132694
b     one    -0.614490
      two    -0.818761
Name: data1, dtype: float64

In [6]:
# use the unstack() function to pivot the data to wide format for presentation purposes
means.unstack()

key2,one,two
key1,,
a,-0.728973,-0.132694
b,-0.614490,-0.818761


In [7]:
# the size() function tells us how many data values belong to each grouping
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

- We can apply iterators to the groups identified by the groupby() function

In [13]:
# iterate over groups defined by the contents of a single column
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.515601 -1.076398
1    a  two -1.268082 -0.236878
4    a  one  0.073821 -2.642625
b
  key1 key2     data1     data2
2    b  one  1.073105  0.845679
3    b  two  0.143901  0.316948


In [12]:
# iterate over groups defined by the contents of two columns, i.e, primary key & secondary key
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.515601 -1.076398
4    a  one  0.073821 -2.642625
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.268082 -0.236878
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.073105  0.845679
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.143901  0.316948


### Limiting the Grouping to Specific Columns

- When grouping data, oftentimes you are interested in grouping the values of specific individual columns rather than all of the columns contained in the data set

In [14]:
# limit the grouping to the column named 'data2'
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.859511
     two  -0.236878
b    one   0.845679
     two   0.316948

In [15]:
# alternatively, save the grouping to a variable name for future use
s_grouped = df.groupby(['key1', 'key2'])['data2']

# now get the mean of the grouping you created
s_grouped.mean()

key1  key2
a     one    -1.859511
      two    -0.236878
b     one     0.845679
      two     0.316948
Name: data2, dtype: float64

### Using a dictionary object to define new grouping categories

- We can use a dictionary object to define groupings that aren't evident in our original data. 


- With a dictionary object, we define a different grouping/subsetting structure on the contents of the DataFrame that differs from the groupings we could create if we were limited to making use of the actual content of the DataFrame (e.g., column indices/labels; row indices/labels; data values).

In [2]:
# define some data to use as a DataFrame
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.767770,-1.270326,1.575059,-0.233593,-1.068810
Steve,-1.720491,-0.930588,-1.974546,0.820645,-0.938253
Wes,-1.405675,NaN,NaN,1.119401,-0.068626
Jim,0.329289,0.049190,0.329772,1.176949,0.155076
Travis,-1.576976,0.793038,-0.242152,1.433289,0.970182


In [3]:
# define a dictionary that assigns alternative grouping labels to
# the content of the columns of the DataFrame
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [4]:
# now apply the grouping specified by the dictionary object to the columns
# of the data frame:
# Note how the new grouping labels "red" and "blue"
# define a new/different aggregation structure on the contents of the DataFrame
# from what we could get if we were limited to making use of the DataFrame's 
# column names for grouping purposes.
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.341465,-3.106906
Steve,-1.153901,-3.589333
Wes,1.119401,-1.474300
Jim,1.506721,0.533554
Travis,1.191138,0.186244


### Using Python Functions to Define The Groupings

- We can use the results of a Python function to define the group mappings. Here's a simple example from the PfDA text: use the len() function to group the data in the 'people' dataframe based on the length of the names used as labels for the row indices

In [5]:
# use the length of the people dataframe's row index labels to group the contents
# of the DataFrame, then calculate a sum total for each of the groupings:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.844156,-1.221136,1.904831,2.062757,-0.982360
5,-1.720491,-0.930588,-1.974546,0.820645,-0.938253
6,-1.576976,0.793038,-0.242152,1.433289,0.970182


### Grouping by Index Levels

- If we have a hierarchical index, we can define our grouping based on a single index level rather than having to make use of the entire hierarchy.

In [6]:
# define a dataframe that has a hierarchical indexing scheme:
# here we have 2 levels for the column indices, with 'cty' serving as 
# the primary key and 'tenor' serving as the secondary key
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.591447 -1.103939  1.361610  0.999606 -0.730185
1      0.889804  0.767115 -0.264501 -1.009897  0.226939
2     -0.003958 -1.434826  0.577169  2.242490  0.167663
3     -0.505994 -1.638297  0.888645 -0.804443 -0.744998

In [8]:
# group by only the 'cty' level of the hierarchical column index
# here we are simply counting the number of data values within each 'cty' grouping
# on a row-by-row basis
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [7]:
# group by only the 'tenor' level of the hierarchical column index
# here we are simply counting the number of data values within each 'tenor' 
# grouping on a row-by-row basis
hier_df.groupby(level='tenor', axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


## Data Aggregation

- The term 'aggregation' refers to any data transformation that produces scalar values from a group of data.

- Examples of aggregation functions in Pandas include __count, sum, mean, median, std, var, min, max, prod, first, last__ and __quantile__.

In [11]:
# reprint the df data frame for ease of reference
df

,key1,key2,data1,data2
0,a,one,-2.700949,0.492007
1,a,two,-0.897015,-1.132052
2,b,one,-0.049058,-0.052414
3,b,two,-0.622501,0.307560
4,a,one,0.481995,-0.005880


In [15]:
# an example of a data aggregation: applying the mean() function to a 
# grouping
grouped = df.groupby('key1')
grouped['data1'].mean()

key1
a   -1.038657
b   -0.335779
Name: data1, dtype: float64

We can also create our own data aggregation functions for use with grouped data. All we need to do is pass a function we've defined to the ".agg()" method associated with a groupby object. Here's an example from the PfDA text:

In [16]:
# define an aggregating function that accepts a groupby object and returns the 
# difference between the max and min values within the group
def peak_to_peak(arr):
    return arr.max() - arr.min()

# pass the function name to the .agg() method of the groupby object we created
# earlier
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.182944,1.624060
b,0.573442,0.359973


### Column-wise Aggregation & Applying Multiple Functions to a Group

- What if you want to aggregate by applying different functions to different groups or specific columns? 


- What if you want to apply __multiple__ functions to a group or column?

In [8]:
# read the tips.csv data set from the PfDA github repo
tips = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv')

# Add a new column of derived data: tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

# display the first 6 rows of the data frame
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [33]:
# we have 244 rows of data in the tips data set
tips.shape

(244, 7)

In [21]:
# group the tips data by day, smoker
grouped = tips.groupby(['day', 'smoker'])

In [24]:
# now lets work with just the tip_pct component of the groupby object
# we've created: 
# compute the mean tip percentage for each day / smoker combination
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

Pass __a list of functions__ to apply to the grouping of the tip_pct data:
note how the output is a data frame that has the results of each
function presented as a separate column

In [25]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [26]:
# apply multiple functions to multiple columns within a grouping
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [27]:
# if we want to examine the results for just a single column of the above:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [28]:
# apply diffent functions to one or more columns: use a dict object
# to define the mappings of columns -> functions
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

### Apply different functions to one or more columns: 

Use a dict object to define the mappings of columns -> functions. This time we want to apply 4 functions to the tip_pct column, so we define a list of function names as the dict value to be associated with the tip_pct dict key:

In [29]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

__Suppress the row index values by passing 'as_index = False'__:

Note how this results in a 'tidy' data set that is free of a hierarchical index.

Compare this output to the cell below where the row index was NOT suppressed

In [32]:
# Suppress the row index values by passing 'as_index = False'
tip_grp = tips.groupby(['day', 'smoker'], as_index=False).mean()
tip_grp

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [31]:
# the same output as above but without the row index suppressed: this 
# output is not 'tidy' due to the hierarchical nature of the row index
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

## Apply: How to use non-aggregating functions with groups ("Split-Apply-Combine")

- The apply() function allows us to invoke non-aggregating functions on individual rows or columns of groupby objects and data frames

In [46]:
# define a function that finds the largest values in a column
# note that this is clearly NOT an aggregating function like mean, max, etc.
# also note that the default column is tip_pct and a default of n=5
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending = True)[-n:]

# find the top 3 tip percentage values in the tips data set
top(tips, n=3)

,total_bill,tip,smoker,day,time,size,tip_pct
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [48]:
# now use the 'top' function to find the top 4 tip percentages for smokers
# and non-smokers. Note how arguments are passed separate from the function
# name, i.e., '(top, n=4)'
tips.groupby('smoker').apply(top, n = 4)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [50]:
# now group the tips data by smoker type & day and find the 2 largest 
# total bills for each smoker type/day combo
tips.groupby(['smoker', 'day']).apply(top, n=2, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [53]:
# to suppress the hierarchical row indexing use the 'group_keys=False' argument
# with the groupby() function
tips.groupby(['smoker', 'day'], group_keys=False).apply(top, n=2, column='total_bill')

,total_bill,tip,smoker,day,time,size,tip_pct
91,22.49,3.50,No,Fri,Dinner,2,0.155625
94,22.75,3.25,No,Fri,Dinner,2,0.142857
59,48.27,6.73,No,Sat,Dinner,4,0.139424
212,48.33,9.00,No,Sat,Dinner,4,0.186220
112,38.07,4.00,No,Sun,Dinner,3,0.105070
156,48.17,5.00,No,Sun,Dinner,6,0.103799
85,34.83,5.17,No,Thur,Lunch,4,0.148435
142,41.19,5.00,No,Thur,Lunch,5,0.121389
90,28.97,3.00,Yes,Fri,Dinner,2,0.103555
95,40.17,4.73,Yes,Fri,Dinner,4,0.117750


### Quantile & Bucket Analysis

- Split data into buckets of equal numeric range using the 'pd.cut()' function


- Split data into buckets of equal item counts using the 'pd.qcut()' function

In [3]:
# create a data frame of 1000 random rows, 2 columns.
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame.head(5)

,data1,data2
0,2.005054,-0.009478
1,-0.092521,0.379442
2,-1.135312,-0.435768
3,-0.731692,0.064878
4,1.390635,1.932808


In [4]:
# split the data1 column into 4 separate bins of equal numeric range
quartiles = pd.cut(frame.data1, 4)
quartiles[:5]

0       (2.0, 3.796]
1    (-1.593, 0.203]
2    (-1.593, 0.203]
3    (-1.593, 0.203]
4       (0.203, 2.0]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.396, -1.593] < (-1.593, 0.203] < (0.203, 2.0] < (2.0, 3.796]]

Now if we want to compute a set of statistics for the 'data2' column relative to the 'data1' cut groupings we derived above, we group __data2__ by the quartile cuts, then apply a user-defined function containing the statistics we want to calculate for each quartile cut:

In [5]:
# define a function that specifies the aggregating functions we want to apply
# to the data2 values contained within each of our data1 quartile cuts
# Note the use of a dict object
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

# group the data2 column's values relative to the data1 quartile cuts we made
# above
grouped = frame.data2.groupby(quartiles)

# apply the get_stats function to the grouped data2 values
# In the results we see the data1 quartile cuts are used as the row index labels
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.396, -1.593]",59.0,2.278537,0.075686,-1.975595
"(-1.593, 0.203]",554.0,2.822868,0.003064,-3.361363
"(0.203, 2.0]",366.0,2.859726,0.087260,-2.783781
"(2.0, 3.796]",21.0,1.354163,-0.012573,-1.337188


In [6]:
# use pd.qcut to get 4 equal size buckets
grouping = pd.qcut(frame.data1, 4, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,250.0,2.822868,0.107036,-3.361363
1,250.0,2.278566,-0.091015,-2.440935
2,250.0,2.859726,0.064132,-2.382291
3,250.0,2.597812,0.071191,-2.783781


## Pivot Tables

- A pivot table is a data summarization tool that aggregates a table of data by one or more keys. The result is a rectangular arrangement of the data with the keys split among the row and column indices of the rectangular object.


- Pandas has a generalized pd.pivot_table() function


- Each DataFrame also has a built-in pivot_table() method

In [62]:
# compute a table of group means arranged by day and smoker
# Note that 'mean()' is the default aggregation type used by the pivot_table()
# function.
# Also note that we could have done this using the groupby() function, e.g.
# tips.groupby(['day', 'smoker']).mean()
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [63]:
# calculate the aggregate means by tip_pct and size + put time, day in row 
# index and smoker in column index
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [8]:
# use 'aggfunc=' argument to specify a function to use other than mean()
# use margins=True to automatically include a summary column containing 
# the results of applying the specified function across all columns.
# The output below is the tabulation of row counts calculated by 'time' and 'smoker'
tips.pivot_table('size', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [70]:
# specify a fill value to use if there are NaN's in your data
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## Cross Tabulation

- A cross tabulation is a type of pivot table that is used to compute group frequencies


- The pd.crosstab() function provides this functionality


An example from the PfDA text:

In [71]:
# define some sample data for cross tabulation
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [72]:
# calculate a crosstab for 'handedness' by 'nationality'
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


## Case Study: 'hflights' Data set

In [3]:
filename = "https://raw.githubusercontent.com/jtopor/DAV-5400/master/Project1/hflights.csv"
df2 = pd.read_csv(filename)
df2.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,...,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted
0,2011,1,1,6,1400.0,1500.0,AA,428,N576AA,60.0,...,-10.0,0.0,IAH,DFW,224,7.0,13.0,0,NaN,0
1,2011,1,2,7,1401.0,1501.0,AA,428,N557AA,60.0,...,-9.0,1.0,IAH,DFW,224,6.0,9.0,0,NaN,0
2,2011,1,3,1,1352.0,1502.0,AA,428,N541AA,70.0,...,-8.0,-8.0,IAH,DFW,224,5.0,17.0,0,NaN,0
3,2011,1,4,2,1403.0,1513.0,AA,428,N403AA,70.0,...,3.0,3.0,IAH,DFW,224,9.0,22.0,0,NaN,0
4,2011,1,5,3,1405.0,1507.0,AA,428,N492AA,62.0,...,-3.0,5.0,IAH,DFW,224,9.0,9.0,0,NaN,0


### Get the average arrival delay for each airline/day of week combo

In [13]:
# get the average arrival delay by airline and day of week
grouped = df2.groupby(['UniqueCarrier', 'DayOfWeek'])

# now lets work with just the ArrDelay component of the groupby object
# we've created: 
# compute the mean arrival delay for each unique carrier / day of week combination
grouped_arrdel = grouped['ArrDelay']

df3 = pd.DataFrame(grouped_arrdel.agg(['mean']))
df3.head(20)

mean
UniqueCarrier DayOfWeek           
AA            1           1.175000
              2          -0.181818
              3           0.328571
              4           5.986111
              5           4.470588
              6          -4.814286
              7          -0.307692
AS            1          29.888889
              2           8.000000
              3           4.875000
              4           9.875000
              5           2.875000
              6          -1.111111
              7           9.888889
B6            1           2.055556
              2          -0.384615
              3         -10.222222
              4          36.000000
              5          23.400000
              6          10.000000

In [6]:
# give the 'mean' column a meaningful label
df3.rename(columns={'mean': 'Average Arrival Delay'}, inplace = True)
df3.head(10)

Average Arrival Delay
UniqueCarrier DayOfWeek                       
AA            1                       1.175000
              2                      -0.181818
              3                       0.328571
              4                       5.986111
              5                       4.470588
              6                      -4.814286
              7                      -0.307692
AS            1                      29.888889
              2                       8.000000
              3                       4.875000

### Get the average arrival delay by UniqueCarrier, Origin and Destination

In [35]:
# get the average arrival delay by airline, origin and destination
grouped = df2.groupby(['UniqueCarrier', 'Origin', 'Dest'])

# now lets work with just the ArrDelay component of the groupby object
# we've created: 
# compute the mean arrival delay for each unique carrier/origin/dest combination
grouped_arrdeldest = grouped['ArrDelay']

df3 = pd.DataFrame(grouped_arrdeldest.agg(['mean'])) 
df3.head(30)


mean
UniqueCarrier Origin Dest           
AA            IAH    DFW    2.133739
                     MIA   -1.274286
AS            IAH    SEA    9.396552
B6            HOU    JFK   11.086022
CO            IAH    ABQ    0.096774
                     ATL   -1.083333
                     AUS    7.581749
                     BHM    9.750000
                     BOS    0.314465
                     BWI   -2.611111
                     CLE    0.049261
                     CLT  -10.500000
                     CMH   -2.727273
                     DCA   -2.954918
                     DEN   11.151515
                     DFW    6.897436
                     DTW    1.902439
                     EGE    1.285714
                     ELP    1.607143
                     EWR    7.752187
                     FLL    0.688623
                     GUC    4.863636
                     HDN   -3.090909
                     HNL   22.738095
                     IAD   -0.674157
                     IND    1.719298
                     LAS    1.673387
                     LAX   11.583333
                     LGA   11.904564
                     MCI    4.214286

### How many flights are there to each destination per unique carrier?

In [15]:
# how many flights to each destination per unique carrier?
hf1 = pd.DataFrame(df2.groupby([df2['UniqueCarrier'], df2['Dest']]).size() )
hf1.head(20)

0
UniqueCarrier Dest     
AA            DFW   342
              MIA   177
AS            SEA    59
B6            JFK   100
CO            ABQ    31
              ATL    63
              AUS   264
              BHM     4
              BOS   164
              BWI   128
              CLE   203
              CLT    31
              CMH    11
              DCA   252
              DEN   297
              DFW    39
              DTW   123
              EGE    35
              ELP    28
              EWR   357

In [25]:
# We can also find these values via a cross tabulation, though the output is not as easy to interpret
pd.crosstab(df2.UniqueCarrier, df2.Dest, margins=True)

Dest,ABQ,AEX,AMA,ASE,ATL,AUS,AVL,BHM,BNA,BOS,...,SMF,SNA,STL,TPA,TUL,TUS,TYS,VPS,XNA,All
UniqueCarrier,,,,,,,,,,,,,,,,,,,,,
AA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,519
AS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,59
B6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100
CO,31,0,0,0,63,264,0,4,0,164,...,100,158,0,163,29,27,0,0,0,6830
DL,0,0,0,0,231,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,258
EV,0,0,0,0,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,106
F9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,62
FL,0,0,0,0,144,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,144
MQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,331


### How many flights to each destination per unique carrier for each day of the week?

In [12]:
# how many flights to each destination per unique carrier for each day of the week?
hf1 = pd.DataFrame(df2.groupby([df2['UniqueCarrier'], df2['Dest'], df2['DayOfWeek']]).size() )
hf1.head(20)

0
UniqueCarrier Dest DayOfWeek    
AA            DFW  1          54
                   2          48
                   3          48
                   4          48
                   5          48
                   6          44
                   7          52
              MIA  1          27
                   2          24
                   3          24
                   4          24
                   5          24
                   6          27
                   7          27
AS            SEA  1           9
                   2           8
                   3           8
                   4           8
                   5           8
                   6           9

### What is the average arrival delay per distance if distance is separated into 3 equal length bins?

In [23]:
# split the Distance column into 3 separate bins (one for each day)
distbins = pd.cut(df2.Distance, 3, precision = 0)
distbins[:10]

0    (123.0, 1386.0]
1    (123.0, 1386.0]
2    (123.0, 1386.0]
3    (123.0, 1386.0]
4    (123.0, 1386.0]
5    (123.0, 1386.0]
6    (123.0, 1386.0]
7    (123.0, 1386.0]
8    (123.0, 1386.0]
9    (123.0, 1386.0]
Name: Distance, dtype: category
Categories (3, interval[float64]): [(123.0, 1386.0] < (1386.0, 2645.0] < (2645.0, 3904.0]]

In [18]:
# define a function that specifies the aggregating functions we want to apply
# to the 'ArrDelay' values contained within each of our Dist binning cuts
# Note the use of a dict object
def get_stats(group):
    return {'Min ArrDelay': group.min(), 'Max ArrDelay': group.max(),
            'Count': group.count(), 'Average ArrDelay': group.mean()}

In [24]:
# group the df2 column's values relative to the Dist binning cuts we made
# above
grouped = df2.ArrDelay.groupby(distbins)

# apply the get_stats function to the grouped data2 values
# In the results we see the data1 quartile cuts are used as the row index labels
grouped.apply(get_stats).unstack()

,Average ArrDelay,Count,Max ArrDelay,Min ArrDelay
Distance,,,,
"(123.0, 1386.0]",4.785782,17879.0,775.0,-49.0
"(1386.0, 2645.0]",8.928453,1817.0,267.0,-43.0
"(2645.0, 3904.0]",22.738095,42.0,326.0,-16.0


### Calculate the Average ArrDelay and DepDelay per UniqueCarrier/Origin/Dest via a Pivot Table

In [37]:
# calc aggregate means for ArrDelay and DepDelay + put UniqueCarrier, Origin and Dest in row 
# index 
df2.pivot_table(['DepDelay', 'ArrDelay'], index=['UniqueCarrier', 'Origin', 'Dest'])

ArrDelay   DepDelay
UniqueCarrier Origin Dest                      
AA            IAH    DFW    2.133739   6.721212
                     MIA   -1.274286   3.528409
AS            IAH    SEA    9.396552   4.559322
B6            HOU    JFK   11.086022  18.234043
CO            IAH    ABQ    0.096774   5.548387
                     ATL   -1.083333   3.216667
                     AUS    7.581749   6.833333
                     BHM    9.750000  -2.000000
                     BOS    0.314465   8.477987
                     BWI   -2.611111   4.642857
                     CLE    0.049261   6.231527
                     CLT  -10.500000  -1.833333
                     CMH   -2.727273   3.454545
                     DCA   -2.954918   3.479508
                     DEN   11.151515  13.902357
                     DFW    6.897436   4.974359
                     DTW    1.902439   5.000000
                     EGE    1.285714   5.485714
                     ELP    1.607143   5.000000
                     EWR    7.752187  16.637681
                     FLL    0.688623   6.844311
                     GUC    4.863636   2.521739
                     HDN   -3.090909   0.911765
                     HNL   22.738095  25.348837
                     IAD   -0.674157   4.311111
                     IND    1.719298   6.228070
                     LAS    1.673387   7.233871
                     LAX   11.583333  14.214744
                     LGA   11.904564  20.702479
                     MCI    4.214286   5.142857
...                              ...        ...
XE            IAH    MCI    6.459184   3.717172
                     MCO   75.000000  70.666667
                     MEM    2.969925   1.992481
                     MFE    5.656716   3.611940
                     MKE   11.211538   6.207547
                     MOB    2.015267   3.587786
                     MSP   12.865385   7.394231
                     MSY    9.000000   2.500000
                     MTJ    8.166667   0.083333
                     OKC    2.885870   3.581522
                     OMA    9.342857   3.778571
                     ORF    4.774194   3.354839
                     PBI    5.641026   0.641026
                     PHX    5.190476   3.809524
                     PIT    0.035714  -0.982456
                     PNS    2.581818   3.378378
                     RDU    2.000000   2.241379
                     RIC    7.650000   8.400000
                     RSW   -0.333333  -1.666667
                     SAT   -2.281690   0.295775
                     SAV    9.788462   8.132075
                     SDF   -2.345794   1.224299
                     SHV    0.971429   3.142857
                     STL   -1.569767   1.709302
                     TPA  -11.000000  -3.666667
                     TUL    2.169154   1.990099
                     TUS    4.581395   6.860465
                     TYS    3.230769   5.410256
                     VPS    1.059701   6.308824
                     XNA    0.397590   1.144578

[201 rows x 2 columns]